Canonical Correlation of Speech Features and Brain Activity

In [7]:
import sklearn.cross_decomposition
print(dir(sklearn.cross_decomposition))


ModuleNotFoundError: No module named 'sklearn'

In [6]:
import scipy.io
import pandas as pd
from sklearn.cross_decomposition import CCA
import statsmodels.formula.api as smf
from statsmodels.stats.mediation import Mediation

ModuleNotFoundError: No module named 'sklearn'

In [ ]:


# Step 1: Load the .mat file
# Load the .mat file containing the ROI data
mat_data = scipy.io.loadmat('path_to/extracted_roi_data.mat')

# Step 2: Convert the .mat structure to a pandas DataFrame
# Initialize an empty list to store rows
rows = []

# Extract the nested structure from the .mat file
extracted_data = mat_data['extracted_data'][0,0]  # Adjust if necessary

# Loop through the structure and extract ROI, Contrast, Participant, and MeanValue
for roi_name in extracted_data.dtype.names:
    roi_data = extracted_data[roi_name][0,0]
    for contrast_name in roi_data.dtype.names:
        contrast_data = roi_data[contrast_name][0,0]
        for participant_name in contrast_data.dtype.names:
            mean_value = contrast_data[participant_name][0,0]
            rows.append([roi_name, contrast_name, participant_name, mean_value])

# Convert to DataFrame
df = pd.DataFrame(rows, columns=['ROI', 'Contrast', 'Participant', 'MeanValue'])

# Step 3: Prepare the data for CCA
# Assume you have separate speech features and brain activity DataFrames
# speech_features_df: DataFrame containing speech features (columns) and participants (rows)
# brain_activity_df: DataFrame containing brain activity measures (columns) and participants (rows)

# Merge the extracted ROI data with your speech features and brain activity data
# Replace 'participant_column' with the actual column name containing participant identifiers in your datasets
merged_df = df.merge(speech_features_df, left_on='Participant', right_on='participant_column')
merged_df = merged_df.merge(brain_activity_df, left_on='Participant', right_on='participant_column')

# Step 4: Perform Canonical Correlation Analysis (CCA)
# Extract the speech features and brain activity data
X = merged_df[speech_features_columns]  # Replace with your actual speech features columns
Y = merged_df[brain_activity_columns]   # Replace with your actual brain activity columns

# Initialize CCA model
cca = CCA(n_components=2)
X_c, Y_c = cca.fit_transform(X, Y)

# Store the canonical variables in the DataFrame for further analysis
merged_df['Canonical_X1'] = X_c[:, 0]
merged_df['Canonical_Y1'] = Y_c[:, 0]

# Step 5: Include the BDI scores in the DataFrame
# Assume 'bdi_scores_df' contains the BDI scores with a column 'Participant' for participant IDs
merged_df = merged_df.merge(bdi_scores_df, on='Participant')

# Step 6: Perform Mediation Analysis
# First model: Canonical X -> BDI (Mediator)
model1 = smf.ols('BDI ~ Canonical_X1', data=merged_df).fit()

# Second model: BDI -> Canonical Y, controlling for Canonical X
model2 = smf.ols('Canonical_Y1 ~ BDI + Canonical_X1', data=merged_df).fit()

# Perform the mediation analysis
med = Mediation(model1, model2, 'Canonical_X1', 'BDI').fit()
print(med.summary())
